# Hyperparameter Tuning using HyperDrive

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [ ]:
from azureml.core import Workspace, Experiment
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform
import os
from azureml.train.hyperdrive.policy import MedianStoppingPolicy
from azureml.train.hyperdrive import uniform, choice
import joblib

## Dataset

TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [ ]:
# Check core SDK version number
print("SDK version:", azureml.core.VERSION)

## Getting available workspace
ws = Workspace.from_config()

experiment_name = 'udacity-project'

exp = Experiment(workspace=ws, name=experiment_name)

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

In [ ]:
## Creating compute cluster
cluster_name = "cpu-cluster"
compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2',max_nodes=4)
compute_target = ComputeTarget.create(ws, cluster_name, compute_config)
compute_target.wait_for_completion(show_output=True, min_node_count = None, timeout_in_minutes=20)
    
#check status of the current cluster
print(compute_target.get_status().serialize())


## Hyperdrive Configuration

TODO: Explain the model you are using and the reason for chosing the different hyperparameters, termination policy and config settings.

In [ ]:
# Logistic Regression needs 2 parameters - 
# C = Regularization Strength (float)
# max_iter = Max iterations (int)
# We use Random Parameter Sampling initially as it supports discrete as well as continuous parameters
# We will try Bayesian Sampling later
# Check initial values for parameters. Current values are used for trial

## Create different params that will be used during training
param_sampling = RandomParameterSampling({
    "C": uniform(1,5),
    "max_iter": choice(range(1, 100))
})

primary_metric_name = "Accuracy"
primary_metric_goal = PrimaryMetricGoal.MAXIMIZE

## Specify early termination policy
early_termination_policy = MedianStoppingPolicy(evaluation_interval=1, delay_evaluation=5)

if "training" not in os.listdir():
    os.mkdir("./training")

## Create a SKLearn estimator for use with train.py
estimator = SKLearn(source_directory = '.',
              entry_script = 'train.py',
              compute_target = compute_target)

## Create a HyperDriveConfig using the estimator, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig(hyperparameter_sampling=param_sampling,
                                     primary_metric_name=primary_metric_name,
                                     primary_metric_goal=primary_metric_goal,
                                     estimator=estimator,
                                     policy=early_termination_policy,
                                     max_total_runs=20)


In [ ]:
## Submit the hyperdrive run to the experiment and show run details with the widget
ws.write_config(path='.azureml/')

hyperdrive_run = exp.submit(hyperdrive_config)

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [ ]:
## Use RunDetails widget to show details of the experiment in progress
RunDetails(hyperdrive_run).show()
hyperdrive_run.wait_for_completion(show_output=True)
assert(hyperdrive_run.get_status() == 'Completed')

## Best Model

TODO: In the cell below, get the best model from the hyperdrive experiments and display all the properties of the model.

In [ ]:
## Get the best run and save the model from that run
hyperdrive_best_run = hyperdrive_run.get_best_run_by_primary_metric()
hyperdrive_best_run_mertics = hyperdrive_best_run.get_metrics()
parameter_values = hyperdrive_best_run.get_details()['runDefinition']['arguments']
print('best run id : ', hyperdrive_best_run.id)
print('best run accuracy :', hyperdrive_best_run_mertics['Accuracy'])
print('C : ', parameter_values[1])
print('max_iter : ', parameter_values[3])

## Save the best model
best_run_model_name = 'Udacity-hyperdrive-model'
best_run_model_path = './outputs/model.joblib'
hyperdrive_best_run.download_file('outputs/model.joblib',best_run_model_path)
model = hyperdrive_best_run.register_model(model_name=best_run_model_name,model_path=best_run_model_path)


## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

TODO: In the cell below, send a request to the web service you deployed to test it.

TODO: In the cell below, print the logs of the web service and delete the service